In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

# Ensure NLTK's tokenizer is ready
nltk.download('punkt')

def extract_summary_from_pdf(text):
    try:
        # Tokenize text into sentences
        sentences = sent_tokenize(text)

        # Choose the first few sentences (e.g., first 3)
        summary = " ".join(sentences[:3])
        return summary if summary else "Summary not available."
    except Exception as e:
        logging.error(f"Error summarizing text: {e}")
        return "Summary not available."


In [ ]:
# Function to extract summary from the PDF content
# def extract_summary_from_pdf(text):
#     try:
#         max_chunk_length = 1024  # HuggingFace models handle up to ~1024 tokens
#         text_chunks = [text[i:i + max_chunk_length] for i in range(0, len(text), max_chunk_length)]

#         summaries = []
#         for chunk in text_chunks:
#             # Set max_length proportional to the chunk length
#             chunk_length = len(chunk.split())
#             max_summary_length = min(150, chunk_length // 2)  # Ensure summary length is not greater than input

#             summary = summarizer(chunk, max_length=max_summary_length, min_length=30, do_sample=False)[0]['summary_text']
#             summaries.append(summary)

#         return " ".join(summaries)
#     except Exception as e:
#         logging.error(f"Error summarizing text: {e}")
#         return "Summary not available."

In [ ]:
def extract_summary_from_pdf(text, keywords):
    try:
        sentences = sent_tokenize(text)
        summary_sentences = [sentence for sentence in sentences if any(keyword in sentence for keyword in keywords)]
        
        # If no sentences match keywords, fall back to first few sentences
        if not summary_sentences:
            summary_sentences = sentences[:3]
        
        summary_text = " ".join(summary_sentences)
        return summary_text if summary_text else "Summary not available."
    except Exception as e:
        logging.error(f"Error summarizing text: {e}")
        return "Summary not available."


In [ ]:
#surya
def classify_and_process_pdf(pdf_file, doc_id):
    metadata, num_pages = extract_pdf_metadata(pdf_file)
    if not metadata:
        return

    if num_pages <= 10:
        category = "Short PDF"
    elif num_pages <= 30:
        category = "Medium PDF"
    else:
        category = "Long PDF"

    logging.info(f"Processing {pdf_file} as {category} with {num_pages} pages.")

    # Check if document exists in MongoDB, if not, insert it
    if collection.find_one({"_id": doc_id}) is None:
        try:
            collection.insert_one({"_id": doc_id, **metadata})
            logging.info(f"Document {doc_id} inserted into MongoDB.")
        except Exception as e:
            logging.error(f"Error inserting document {doc_id}: {e}")
            return

    # Extract text from PDF
    text = extract_text_from_pdf(pdf_file)

    if not text:
        logging.error(f"No text extracted from {pdf_file}. Skipping summarization and keyword extraction.")
        return

    # Perform summarization using NLP
    summary = extract_summary_from_pdf(text)

    # Perform keyword extraction using NER
    keywords = extract_keywords_from_text(text)

    # Update MongoDB with summary and keywords
    update_mongo_with_summary(doc_id, summary, keywords)

: 